In [1]:
%pip install h5py tqdm torch fvcore

Defaulting to user installation because normal site-packages is not writeable
  Using cached h5py-3.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached torch-2.1.2-cp38-cp38-manylinux1_x86_64.whl (670.2 MB)
Processing /home/cs7561/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc/fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached triton-2.1.0-0-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-

In [3]:
import h5py
from tqdm import tqdm
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'tqdm'

In [26]:
dataset_path = "/scratch/work/ptg/EPIC-KITCHENS/EPIC_audio.hdf5"
audio_dataset = h5py.File(dataset_path, "r")

In [27]:
audio_dataset['P01_01'].shape

(39651328,)

In [28]:
samples = audio_dataset['P01_01'][()]
samples.shape

(39651328,)

In [29]:
SAMPLE_RATE = 24_000
WINDOW_SIZE = 2 # In seconds
HOP_SIZE = 0.5
HOP_SIZE_SAMPLES = int(round(HOP_SIZE * SAMPLE_RATE))
SAMPLE_DURATION = 1.999
SAMPLES_PER_WINDOW = int(round(SAMPLE_RATE * WINDOW_SIZE))

In [31]:
video_durations_path = "/scratch/cs7561/auditory-slow-fast/data/epic-kitchens-100-annotations/EPIC_100_video_info.csv"
video_durations = pd.read_csv(video_durations_path)
video_durations["n_samples"] = [self.audio_records[vid_id].shape[0] for vid_id in video_durations["video_id"]]



#video_durations["n_samples"] = video_durations["duration"] * SAMPLE_RATE
#video_durations["n_samples"] = video_durations["n_samples"].astype(int)
video_durations

NameError: name 'self' is not defined

In [15]:
data_points = {}
for i, video in tqdm(video_durations.iterrows(), unit="video", total=video_durations.shape[0]):    
    data_points[video.video_id] = []
    start = 0
    end = WINDOW_SIZE # TODO: Check if that is the right logic
    while start < video.duration:
        end = min(end, video.duration) # TODO: should we keep this or 0-pad instead?

        data_points[video.video_id].append((start, end))
        start += HOP_SIZE
        end = start + WINDOW_SIZE

100%|██████████| 700/700 [00:16<00:00, 43.40video/s]


In [16]:
sum([len(l) for l in data_points.values()])

720546

In [17]:
import datetime

datetime.datetime.min

datetime.datetime(1, 1, 1, 0, 0)

In [49]:
path = "/scratch/cs7561/auditory-slow-fast/data/epic-kitchens-100-annotations/EPIC_100_validation.pkl"
file_df = pd.read_pickle(path)

In [50]:
file_df.head(40)

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,
P01_11_0,P01,P01_11,00:00:00.560,00:00:00.00,00:00:01.89,1,113,take plate,take,0,plate,2,[plate],[2]
P01_11_1,P01,P01_11,00:00:01.700,00:00:01.56,00:00:02.45,93,147,put down plate,put-down,1,plate,2,[plate],[2]
P01_11_10,P01,P01_11,00:00:48.500,00:00:49.15,00:00:50.95,2949,3057,take paper,take,0,paper,49,[paper],[49]
P01_11_100,P01,P01_11,00:05:27.840,00:05:27.28,00:05:31.97,19636,19918,wash cloth,wash,2,cloth,17,[cloth],[17]
P01_11_101,P01,P01_11,00:05:26.840,00:05:27.37,00:05:29.86,19642,19791,take cloth,take,0,cloth,17,[cloth],[17]
P01_11_102,P01,P01_11,00:05:32.500,00:05:30.62,00:05:35.02,19837,20101,squeeze cloth,squeeze,18,cloth,17,[cloth],[17]
P01_11_103,P01,P01_11,00:05:37.000,00:05:35.56,00:05:41.14,20133,20468,wipe counter,wipe,2,counter,42,[counter],[42]
P01_11_104,P01,P01_11,00:05:41.590,00:05:41.24,00:05:48.23,20474,20893,wipe sink,wipe,2,sink,63,[sink],[63]
P01_11_105,P01,P01_11,00:05:50.099,00:05:48.67,00:05:56.10,20920,21366,continue wiping sink,wipe,2,sink,63,[sink],[63]


In [54]:


file_df[file_df["video_id"] == "P32_08"]
file_df["video_id"].unique()

array(['P01_11', 'P01_12', 'P01_13', 'P01_14', 'P01_15', 'P02_12',
       'P02_13', 'P02_14', 'P02_15', 'P03_21', 'P03_22', 'P03_23',
       'P03_24', 'P03_25', 'P03_26', 'P04_24', 'P04_25', 'P04_26',
       'P04_27', 'P04_28', 'P04_29', 'P04_30', 'P04_31', 'P04_32',
       'P04_33', 'P05_07', 'P05_09', 'P06_10', 'P06_11', 'P06_12',
       'P06_13', 'P06_14', 'P07_12', 'P07_13', 'P07_14', 'P07_15',
       'P07_16', 'P07_17', 'P07_18', 'P08_09', 'P08_10', 'P08_14',
       'P08_15', 'P08_16', 'P08_17', 'P09_07', 'P09_08', 'P10_03',
       'P11_17', 'P11_18', 'P11_19', 'P11_20', 'P11_21', 'P11_22',
       'P11_23', 'P11_24', 'P12_03', 'P12_08', 'P13_01', 'P13_02',
       'P13_03', 'P14_06', 'P14_08', 'P15_04', 'P15_05', 'P15_06',
       'P16_04', 'P17_02', 'P18_01', 'P18_02', 'P18_03', 'P18_04',
       'P18_05', 'P18_06', 'P18_07', 'P18_08', 'P18_09', 'P18_10',
       'P18_11', 'P18_12', 'P19_05', 'P19_06', 'P20_05', 'P20_06',
       'P20_07', 'P21_02', 'P22_01', 'P22_02', 'P22_03', 'P22_

In [62]:
from utils import timestamp_to_sec

In [63]:
file_df["start_s"] = file_df["start_timestamp"].map(timestamp_to_sec)
file_df["stop_s"] = file_df["stop_timestamp"].map(timestamp_to_sec)

In [70]:
video_durations[video_durations["video_id"].isin(file_df["video_id"].unique())].sort_values(by=["duration"], ascending=True)

,video_id,duration,fps,resolution
104,P03_26,11.127783,59.94006,1920x1080
515,P26_30,15.281933,59.94006,1920x1080
585,P28_23,25.158467,59.94006,1920x1080
658,P32_07,28.978950,59.94006,1920x1080
40,P02_13,29.512817,59.94006,1920x1080
...,...,...,...,...
13,P01_14,1352.585217,59.94006,1920x1080
336,P12_03,1465.865050,59.94006,1280x720
611,P29_05,1821.736567,59.94006,1920x1080
299,P10_03,1842.442917,59.94006,1920x1080


In [53]:
len(file_df["video_id"].unique())

138

In [72]:
from typing import Optional

def find(video_id: str, mid: Optional[float] = None) -> pd.DataFrame:
    if mid is not None:
        return file_df[file_df["video_id"] == video_id][(file_df["start_s"] <= mid) & (mid <= file_df["stop_s"])]
    else:
        return file_df[file_df["video_id"] == video_id]

In [73]:
find("P03_26")

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes,start_s,stop_s
narration_id,,,,,,,,,,,,,,,,
P03_26_0,P03,P03_26,00:00:04.420,00:00:00.00,00:00:05.13,1,307,put plates on the side,put-on,1,plate,2,"[plate, side]","[2, 42]",0.00,5.13
P03_26_1,P03,P03_26,00:00:05.610,00:00:06.61,00:00:07.64,396,458,open fridge,open,3,fridge,12,[fridge],[12],6.61,7.64
P03_26_2,P03,P03_26,00:00:09.250,00:00:07.24,00:00:09.40,434,564,put grapes in fridge,put-in,5,grape,202,"[grape, fridge]","[202, 12]",7.24,9.40
P03_26_3,P03,P03_26,00:00:10.639,00:00:09.00,00:00:09.98,540,598,close fridge,close,4,fridge,12,[fridge],[12],9.00,9.98


In [74]:
find("P03_26", 8.0)

<ipython-input-72-3afa6350c87e>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return file_df[file_df["video_id"] == video_id][(file_df["start_s"] <= mid) & (mid <= file_df["stop_s"])]


,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes,start_s,stop_s
narration_id,,,,,,,,,,,,,,,,
P03_26_2,P03,P03_26,00:00:09.250,00:00:07.24,00:00:09.40,434,564,put grapes in fridge,put-in,5,grape,202,"[grape, fridge]","[202, 12]",7.24,9.4


In [75]:
find("P02_12", timestamp_to_sec("00:05:43.000000"))

<ipython-input-72-3afa6350c87e>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return file_df[file_df["video_id"] == video_id][(file_df["start_s"] <= mid) & (mid <= file_df["stop_s"])]


,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes,start_s,stop_s
narration_id,,,,,,,,,,,,,,,,
P02_12_108,P02,P02_12,00:05:22.789,00:05:18.75,00:05:45.70,19125,20742,knead dough,knead,51,dough,25,[dough],[25],318.75,345.70
P02_12_110,P02,P02_12,00:05:12.240,00:05:40.40,00:05:45.01,20424,20700,knead dough,knead,51,dough,25,[dough],[25],340.40,345.01


In [76]:
file_df.shape[0]

9668